## Neuro File Transfer (.ps1)

VERSION: `Neuro_Psych_Battery_Auto_Sync_General`

### Advanced Explanation

The script, written in PowerShell, is designed for transferring newly created directories from the local machine to a specified remote server. It starts by obtaining the current date and the user's username. The user is prompted to confirm or specify the remote destination path, preformatted as `/home/$username/scratch/Actual_Final_Neuropsych_Battery/`.

The script identifies directories in the current location created within the last 24 hours using `Get-ChildItem -Directory` combined with a `Where-Object` filter. For each identified directory, it calculates its size in kilobytes and prompts the user for confirmation to transfer to the server. The transfer is conducted using `scp` (Secure Copy Protocol), with the success of each operation logged. The script counts and lists the names of successfully transferred directories, providing feedback once the process is complete. It concludes by waiting for user input to close.

### User-Friendly Explanation

This script is a convenient tool for backing up recent folders to a server. It starts by asking for your username and confirming the destination path on the server. Then, it looks for any folders in your current location that were created in the last day, showing their sizes in kilobytes.

For each new folder, the script asks if you want to transfer it to the server. If you agree, it securely sends the folder to the server, keeping track of each successful transfer. At the end, it reports how many folders were transferred and their names. The script waits for you to press Enter before closing, ensuring you're informed about the completion of the backup process.

### Oversimplified Explanation

This should be kept and run where NeuroPsych Battery session folders are backed up. It will send new ones from the local device to your scratch folder on Compute Canada. 

### Note
    
Track meta data before sending and compare it to the output of the check scripts. 

In [ ]:
$currentDate = Get-Date

$username = Read-Host "Please enter your username"

$remotePathBase = "/home/$username/scratch/Actual_Final_Neuropsych_Battery/"
Write-Output "The constructed remote path is: $remotePathBase"
$pathConfirmation = Read-Host "Is this path correct? [Y/N]"

if ($pathConfirmation -ne "Y") {
    $remotePathBase = Read-Host "Please enter the full correct remote path"
}

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name
    $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1KB

    $userInput = Read-Host "Do you want to push $($dir.Name) [Size: $($directorySize) KB] to the server at `"$remotePath`"? [Y/N]"

    if ($userInput -eq "Y") {

        scp -r $dir.FullName "$username@graham.computecanada.ca:`"$remotePath`""
        if ($?) {
            $transferredCount++
            $transferredDirectories += $dir.Name
            Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
        } else {
            Write-Warning "Failed to transfer directory: $dir"
        }
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}

Read-Host -Prompt "Press Enter to Close"


VERSION: `NeuroPsych_Battery_Auto_Sync_(name).ps1`

In [ ]:
$currentDate = Get-Date

$recentDirectories = Get-ChildItem -Directory | Where-Object { ($currentDate - $_.CreationTime).TotalHours -le 24 }

$transferredCount = 0
$transferredDirectories = @()

$remotePathBase = "/home/samhsu03/scratch/Actual_Final_Neuropsych_Battery/"

foreach ($dir in $recentDirectories) {
    $remotePath = $remotePathBase + $dir.Name

    scp -r $dir.FullName "samhsu03@graham.computecanada.ca:`"$remotePath`""
    if ($?) {
        $transferredCount++
        $transferredDirectories += $dir.Name
        
        $directorySize = (Get-ChildItem $dir.FullName -Recurse | Measure-Object -Property Length -Sum).Sum / 1MB
        Write-Output ("Directory '{0}' transferred. Size: {1:N2} MB" -f $dir.Name, $directorySize)
    } else {
        Write-Warning "Failed to transfer directory: $dir"
    }
}

Write-Output "Transfer process complete."
Write-Output "$transferredCount directory(ies) were transferred:"
$transferredDirectories | ForEach-Object {
    Write-Output $_
}